In [1]:
import json
import csv
from pprint import pprint
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [2]:
path = '/Users/stylshreyeshi/Desktop/location.json'
with open(path) as f:
    data = json.load(f)

In [3]:
df = pd.DataFrame.from_dict(data, orient='columns')
df.head()

,locations
0,"{'timestampMs': '1543320317919', 'latitudeE7':..."
1,"{'timestampMs': '1543319700075', 'latitudeE7':..."
2,"{'timestampMs': '1543319641689', 'latitudeE7':..."
3,"{'timestampMs': '1543319100119', 'latitudeE7':..."
4,"{'timestampMs': '1543318465263', 'latitudeE7':..."


In [ ]:
df = pd.read_json((df['locations']).to_json(), orient='index')
df.head()

In [ ]:
df['activity'].fillna(value='-',inplace=True)
df['activity'] = df['activity'].apply(lambda x: "".join(map(str,x)) )

In [ ]:
df['activity_timestampMs'] = df['activity'].apply(lambda x: x[x.find('timestampMs')+15:30])
df['activity_data'] = df['activity'].apply(lambda x: x[x.find('activity')+22:x.find('confidence')-4])
df['confidence'] = df['activity'].apply(lambda x: x[x.find('confidence')+13:x.find('}')])
df.head()

In [ ]:
from datetime import datetime
df['activity_timestampMs'] = np.where(df['activity_timestampMs']=='',0,pd.to_numeric(df['activity_timestampMs']))
df['activity_timestampMs'] = df['activity_timestampMs'].apply(lambda x: datetime.utcfromtimestamp(int(x/1000)).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
df['latitudeE7'] = df['latitudeE7'].apply(lambda x: int(x)/10000000)
df['longitudeE7'] = df['longitudeE7'].apply(lambda x: int(x)/10000000)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
from shapely.geometry import Point, polygon
%matplotlib inline
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitudeE7, df.latitudeE7))
gdf.head()

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world.plot(color='white', edgecolor='black', figsize=(15,10))
gdf.plot(ax=ax, color='red')
plt.show()

In [ ]:
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
m = folium.Map([28.48, 77.07], zoom_start=12)
df1 = df[(df['timestampMs'].dt.year==2018) & (df['timestampMs'].dt.month==4) & (df['timestampMs'].dt.day<6)]
for index, row in df1.iterrows():
    folium.CircleMarker([row['latitudeE7'], row['longitudeE7']],
                        radius=0.0001,
                        fill_color="#3db7e4", # divvy color
                       ).add_to(m)
point_ = df1[['latitudeE7', 'longitudeE7']].as_matrix()
m.add_child(plugins.HeatMap(point_, radius=15))
m

In [ ]:
df.head()

In [ ]:
import reverse_geocoder as rg 

def reverseGeocode(coordinates): 
    result = ((rg.search(coordinates))[0])['name'] 
    z = result[0]
    return (z['name'])

In [ ]:
df2 = df.copy()

In [ ]:
#Working on df copy df2
df2['coordinates'] = list(zip(df2.latitudeE7, df2.longitudeE7))

In [ ]:
df2 = df2.sa
df2['City'] = df2['coordinates'].apply(lambda x: ((rg.search(coordinates))[0])['name'])